### Try to make memory

In [ ]:
import os
import time
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client
client = AzureOpenAI(
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

def get_user_input():
    """Get initial user input for FATO creation"""
    return input("Describe the FATO in natural language (e.g., 'Create a FATO JSON for...'):\n")

def generate_fato_json(user_input):
    """Generate initial FATO JSON based on user input"""
    function_def = [
        {
            "name": "generate_fato_layout",
            "description": "Generate detailed FATO layout from helicopter and location data",
            "parameters": {
                "type": "object",
                "properties": {
                    # ... (keep your existing properties here) ...
                    "aircraft": {"type": "string"},
                    "fato_length": {"type": "number"},
                    "fato_width": {"type": "number"},
                    "fato_height": {"type": "number"},
                    "sides": {"type": "number"},
                    "diameter": {"type": "number"},
                    "marking_thickness": {"type": "number"},
                    "rotation": {"type": "number"},
                    "isVisible": { "type": "boolean" },
                    "transparency": {"type": "number"},
                    "elevation": {"type": "number"},
                    "shape_type": {
                    "type": "string",
                    "enum": ["Circle", "Polygon", "Rectangle"]
                    },
                    "position": {
                    "type": "array",
                    "items": {"type": "number"},
                    "minItems": 2,
                    "maxItems": 2
                    },
                    "layerName": { "type": "string" },
                    "netHeight": { "type": "number" },
                    "curveAngle": { "type": "number" },
                    "dashLength": { "type": "number" },
                    "lightColor": { "type": "string" },
                    "lightScale": { "type": "number" },
                    "multiplier": { "type": "number" },
                    "lightHeight": { "type": "number" },
                    "lightRadius": { "type": "number" },
                    "markingType": { "type": "string" },
                    "dashDistance": { "type": "number" },
                    "markingColor": { "type": "string" },
                    "lightCategory": { "type": "boolean" },
                    "lightDistance": { "type": "number" },
                    "scaleCategory": { "type": "boolean" },
                    "textureScaleU": { "type": "number" },
                    "textureScaleV": { "type": "number" },
                    "vertipadColor": { "type": "string" },
                    "offsetDistance": { "type": "number" },
                    "safetyAreaType": { "type": "string" },
                    "safetyNetColor": { "type": "string" },
                    "safetyNetScaleU": { "type": "number" },
                    "safetyNetScaleV": { "type": "number" },
                    "aircraftCategory": { "type": "boolean" },
                    "markingsCategory": { "type": "boolean" },
                    "safetyNetCategory": { "type": "boolean" },
                    "safetyAreaCategory": { "type": "boolean" },
                    "safetyNetTransparency": { "type": "number" }
                },
                "required": [
                    "aircraft", "fato_length", "fato_width", "fato_height", "sides",
                    "diameter", "marking_thickness", "rotation", "transparency",
                    "shape_type", "position"
                ]
            }
        }
    ]

    response = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
        messages=[{"role": "user", "content": user_input}],
        functions=function_def,
        function_call={"name": "generate_fato_layout"}
    )

    function_args = json.loads(response.choices[0].message.function_call.arguments)
    
    # Set defaults
    if "elevation" not in function_args or function_args["elevation"] in (None, 0):
        function_args["elevation"] = 20.0

    fato_id = f"FATO-{int(time.time() * 1000)}"
    
    # Build JSON (keep your existing structure)
    fato_json = {
        "id": fato_id,
        # ... (rest of your JSON structure) ...
        "groupId": None,
        "position": function_args["position"],  # [lon, lat]
        "isVisible": True,
        "layerName": "GEOMETRY_FATO",
        "layerOrder": 0,
        "dimensions": {
        "sides": max(3, min(function_args["sides"], 70)),
        "width": max(1, min(function_args["fato_width"], 300)),
        "dValue": 10,
        "height": max(1, min(function_args["fato_height"], 300)), # Thickness        
        "length": max(1, min(function_args["fato_length"], 300)),
        "aircraft": function_args["aircraft"],
        "diameter": max(1.0, min(function_args["diameter"], 200.0)),
        "rotation": max(0, min(function_args["rotation"], 359)),
        "isVisible": True,
        "layerName": "GEOMETRY_FATO",
        "netHeight": 15,
        "shapeType": function_args["shape_type"],
        "baseHeight": max(0.0, min(function_args["elevation"], 1000.0)), #Elevation
        "curveAngle": 45,
        "dashLength": 1,
        "lightColor": "white",
        "lightScale": 1,
        "multiplier": 1.5,
        "lightHeight": 0.2,
        "lightRadius": 0.3,
        "markingType": "dashed",
        "dashDistance": 1,
        "markingColor": "white",
        "transparency": max(0, min(function_args["transparency"], 1)),
        "lightCategory": False,
        "lightDistance": 1,
        "scaleCategory": False,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "vertipadColor": "#808080",
        "offsetDistance": 3,
        "safetyAreaType": "multiplier",
        "safetyNetColor": "#FF0000",
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": False,
        "markingThickness": max(0.01, min(function_args["marking_thickness"], 5.0)),
        "markingsCategory": False,
        "safetyNetCategory": False,
        "safetyAreaCategory": False,
        "safetyNetTransparency": 0.5
    }
}
    
    return fato_json

def ask_for_changes(fato_json):
    """Ask user if they want to make changes and process them"""
    while True:
        print("\nGenerated FATO JSON:")
        print(json.dumps(fato_json, indent=2))
        
        response = input("\nWould you like to make any changes? (yes/no): ").lower()
        
        if response not in ['yes', 'y']:
            break
            
        change_request = input("What would you like to change? (e.g., 'Change width to 50'): ")
        
        # Get the current values as context
        current_values = "\nCurrent values:\n"
        for key, value in fato_json['dimensions'].items():
            if isinstance(value, (str, int, float, bool)):
                current_values += f"{key}: {value}\n"
        
        # Ask AI to interpret the change
        messages = [
            {"role": "system", "content": "You help modify JSON structures. Only respond with valid JSON patches."},
            {"role": "user", "content": current_values + f"\nChange request: {change_request}"}
        ]
        
        try:
            response = client.chat.completions.create(
                model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
                messages=messages,
                temperature=0.3
            )
            
            # Apply changes (simple implementation - you might want more robust patching)
            change = response.choices[0].message.content
            if change.startswith('{') and change.endswith('}'):
                changes = json.loads(change)
                for key, value in changes.items():
                    if key in fato_json['dimensions']:
                        fato_json['dimensions'][key] = value
                        print(f"Updated {key} to {value}")
                    else:
                        print(f"Warning: {key} is not a valid field")
            else:
                print("Couldn't interpret the changes")
                
        except Exception as e:
            print(f"Error processing changes: {e}")
    
    return fato_json

def main():
    """Main execution flow"""
    user_input = get_user_input()
    fato_json = generate_fato_json(user_input)
    final_json = ask_for_changes(fato_json)
    
    print("\nFinal FATO JSON:")
    print(json.dumps(final_json, indent=2))

if __name__ == "__main__":
    main()

Describe the FATO in natural language (e.g., 'Create a FATO JSON for...'):
 create a fato width and height of 30 and rotate 30 deg



Generated FATO JSON:
{
  "id": "FATO-1750331958377",
  "groupId": null,
  "position": [
    0,
    0
  ],
  "isVisible": true,
  "layerName": "GEOMETRY_FATO",
  "layerOrder": 0,
  "dimensions": {
    "sides": 3,
    "width": 30,
    "dValue": 10,
    "height": 1,
    "length": 30,
    "aircraft": "Default",
    "diameter": 1.0,
    "rotation": 30,
    "isVisible": true,
    "layerName": "GEOMETRY_FATO",
    "netHeight": 15,
    "shapeType": "Rectangle",
    "baseHeight": 20.0,
    "curveAngle": 45,
    "dashLength": 1,
    "lightColor": "white",
    "lightScale": 1,
    "multiplier": 1.5,
    "lightHeight": 0.2,
    "lightRadius": 0.3,
    "markingType": "dashed",
    "dashDistance": 1,
    "markingColor": "white",
    "transparency": 0,
    "lightCategory": false,
    "lightDistance": 1,
    "scaleCategory": false,
    "textureScaleU": 1,
    "textureScaleV": 1,
    "vertipadColor": "#808080",
    "offsetDistance": 3,
    "safetyAreaType": "multiplier",
    "safetyNetColor": "#FF0000


Would you like to make any changes? (yes/no):  udpate sides to 5



Final FATO JSON:
{
  "id": "FATO-1750331958377",
  "groupId": null,
  "position": [
    0,
    0
  ],
  "isVisible": true,
  "layerName": "GEOMETRY_FATO",
  "layerOrder": 0,
  "dimensions": {
    "sides": 3,
    "width": 30,
    "dValue": 10,
    "height": 1,
    "length": 30,
    "aircraft": "Default",
    "diameter": 1.0,
    "rotation": 30,
    "isVisible": true,
    "layerName": "GEOMETRY_FATO",
    "netHeight": 15,
    "shapeType": "Rectangle",
    "baseHeight": 20.0,
    "curveAngle": 45,
    "dashLength": 1,
    "lightColor": "white",
    "lightScale": 1,
    "multiplier": 1.5,
    "lightHeight": 0.2,
    "lightRadius": 0.3,
    "markingType": "dashed",
    "dashDistance": 1,
    "markingColor": "white",
    "transparency": 0,
    "lightCategory": false,
    "lightDistance": 1,
    "scaleCategory": false,
    "textureScaleU": 1,
    "textureScaleV": 1,
    "vertipadColor": "#808080",
    "offsetDistance": 3,
    "safetyAreaType": "multiplier",
    "safetyNetColor": "#FF0000",
 